# Model Dev for Detection Issue

## Import

In [ ]:
import pandas as pd
from math import isclose
import json
import os
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dask.dataframe as dd
import pyarrow
print(pyarrow.__version__)
from dask_geopandas import from_geopandas
import geopandas as gpd
from shapely.geometry import Point
import pickle  # Import the pickle module

# Fonction utils

In [22]:
# Fonction pour charger les données JSON
def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

In [23]:
trafic_df = pd.read_csv("./freq_arret/2013-2021_trafic.csv")
trafic_df


,Rang,Réseau,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,Ville,Arrondissement pour Paris,Year
0,12,Métro,OPERA,12269711,3,7,8,NaN,NaN,Paris,9.0,2013
1,13,Métro,HOTEL DE VILLE,11953352,1,11,NaN,NaN,NaN,Paris,4.0,2013
2,19,Métro,NATION,9541406,1,2,6,9.0,NaN,Paris,12.0,2013
3,25,Métro,SAINT-MICHEL,8797595,4,NaN,NaN,NaN,NaN,Paris,5.0,2013
4,20,Métro,PALAIS-ROYAL,9385220,1,7,NaN,NaN,NaN,Paris,1.0,2013
...,...,...,...,...,...,...,...,...,...,...,...,...
3320,39,RER,FONTENAY-SOUS-BOIS,2121720,A,NaN,NaN,NaN,NaN,Fontenay-sous-Bois,NaN,2021
3321,40,RER,SUCY-BONNEUIL,1993503,A,NaN,NaN,NaN,NaN,Sucy-en-Brie,NaN,2021
3322,58,RER,LE GUICHET,530706,B,NaN,NaN,NaN,NaN,Orsay,NaN,2021
3323,65,RER,PARC DE SCEAUX,383626,B,NaN,NaN,NaN,NaN,Antony,NaN,2021


In [24]:
df  = pd.read_csv("./freq_arret/arrets.csv" , sep=';')



# Afficher les premières lignes pour vérifier 
df.head()

,ArRId,ArRVersion,ArRCreated,ArRChanged,ArRName,ArRType,ArRPublicCode,ArRXEpsg2154,ArRYEpsg2154,ZdAId,ArRGeopoint,ArRTown,ArRPostalRegion,ArRAccessibility,ArRAudibleSignals,ArRVisualSigns,ArRFareZone
0,14970,1250460-1533057,2014-12-29T00:00:00+01:00,2023-01-09T09:47:08+01:00,Pasteur,bus,-,671805,6843819,42499,"48.693904034797036, 2.61692901241064",Brie-Comte-Robert,77053,partial,unknown,unknown,5
1,14986,1238023-1238017,2014-12-29T00:00:00+01:00,2023-01-09T09:47:08+01:00,Réveillon Écoles,bus,-,670426,6848172,44638,"48.73299424266356, 2.597892392259737",Lésigny,77249,unknown,unknown,unknown,5
2,14987,1238019-1238017,2014-12-29T00:00:00+01:00,2023-01-09T09:47:08+01:00,Réveillon Écoles,bus,-,670418,6848213,44638,"48.73336263308142, 2.5977807812829954",Lésigny,77249,unknown,unknown,unknown,5
3,14954,886152-886163,2014-12-29T00:00:00+01:00,2023-01-09T09:47:08+01:00,Les Myosotis,bus,-,671673,6843457,426937,"48.690646829044304, 2.6151723319505265",Brie-Comte-Robert,77053,partial,unknown,unknown,5
4,14968,1440816-1440818,2014-12-29T15:31:51+01:00,2023-01-09T09:47:08+01:00,Parc Centre Commercial,bus,-,670959,6848670,55149,"48.737497476060646, 2.6051054273842142",Lésigny,77249,unknown,unknown,unknown,5


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38025 entries, 0 to 38024
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ArRId              38025 non-null  int64 
 1   ArRVersion         38025 non-null  object
 2   ArRCreated         37952 non-null  object
 3   ArRChanged         38025 non-null  object
 4   ArRName            38025 non-null  object
 5   ArRType            38025 non-null  object
 6   ArRPublicCode      38025 non-null  object
 7   ArRXEpsg2154       38025 non-null  int64 
 8   ArRYEpsg2154       38025 non-null  int64 
 9   ZdAId              38025 non-null  int64 
 10  ArRGeopoint        38025 non-null  object
 11  ArRTown            38025 non-null  object
 12  ArRPostalRegion    38025 non-null  int64 
 13  ArRAccessibility   38025 non-null  object
 14  ArRAudibleSignals  38025 non-null  object
 15  ArRVisualSigns     38025 non-null  object
 16  ArRFareZone        36087 non-null  objec

#### analyse 

je remarque correlation de nom entre station et ARnam afin de merged les 2 df

In [4]:

# Charger les DataFrames
trafic_df = pd.read_csv("./freq_arret/2013-2021_trafic.csv")
arrets_df = pd.read_csv("./freq_arret/arrets.csv", sep=';')

# Transformer les colonnes 'Station' et 'ArRName' en minuscules
trafic_df['Station'] = trafic_df['Station'].str.lower()
arrets_df['ArRName'] = arrets_df['ArRName'].str.lower()

# Effectuer la jointure sur les colonnes 'Station' et 'ArRName'
merged_df = pd.merge(trafic_df, arrets_df, left_on='Station', right_on='ArRName', how='inner')

# Afficher les premières lignes du DataFrame fusionné pour vérifier
print("Merged DataFrame:")
print(merged_df[['Station', 'ArRName']].head())

# Vérifier quelques valeurs spécifiques
sample_stations = ['opera', 'hotel de ville']  # Par exemple, en minuscules
sample_arrets = merged_df[merged_df['Station'].isin(sample_stations)]
merged_df


Merged DataFrame:
  Station ArRName
0  nation  nation
1  nation  nation
2  nation  nation
3  nation  nation
4  nation  nation


,Rang,Réseau,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,Ville,...,ArRXEpsg2154,ArRYEpsg2154,ZdAId,ArRGeopoint,ArRTown,ArRPostalRegion,ArRAccessibility,ArRAudibleSignals,ArRVisualSigns,ArRFareZone
0,19,Métro,nation,9541406,1,2,6,9.0,NaN,Paris,...,655661,6861104,462358,"48.84845279706631, 2.3957988351125254",Paris 11e,75111,false,true,true,1
1,19,Métro,nation,9541406,1,2,6,9.0,NaN,Paris,...,655664,6861063,462358,"48.848084289960305, 2.3958439887237373",Paris 12e,75112,false,true,true,1
2,19,Métro,nation,9541406,1,2,6,9.0,NaN,Paris,...,655670,6861104,462358,"48.848453436965976, 2.395925559927623",Paris 11e,75111,false,true,true,1
3,19,Métro,nation,9541406,1,2,6,9.0,NaN,Paris,...,655672,6861152,462358,"48.848885218403474, 2.3959437210498136",Paris 11e,75111,false,true,true,1
4,19,Métro,nation,9541406,1,2,6,9.0,NaN,Paris,...,655671,6861073,473875,"48.84817470172562, 2.3959383298941117",Paris 12e,75112,unknown,unknown,unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16487,65,RER,parc de sceaux,383626,B,NaN,NaN,NaN,NaN,Antony,...,649288,6852420,43177,"48.769885381618195, 2.309994431722374",Antony,92002,partial,unknown,unknown,NaN
16488,65,RER,parc de sceaux,383626,B,NaN,NaN,NaN,NaN,Antony,...,648780,6853370,57500,"48.77838909051428, 2.3029672849882807",Sceaux,92071,partial,unknown,unknown,3
16489,65,RER,parc de sceaux,383626,B,NaN,NaN,NaN,NaN,Antony,...,649285,6852383,43177,"48.769552376312525, 2.309956653435713",Antony,92002,unknown,unknown,unknown,3
16490,65,RER,parc de sceaux,383626,B,NaN,NaN,NaN,NaN,Antony,...,649279,6852385,43177,"48.769569891830265, 2.3098747819817276",Antony,92002,unknown,unknown,unknown,3


In [5]:
merged_df.columns

Index(['Rang', 'Réseau', 'Station', 'Trafic', 'Correspondance_1',
       'Correspondance_2', 'Correspondance_3', 'Correspondance_4',
       'Correspondance_5', 'Ville', 'Arrondissement pour Paris', 'Year',
       'ArRId', 'ArRVersion', 'ArRCreated', 'ArRChanged', 'ArRName', 'ArRType',
       'ArRPublicCode', 'ArRXEpsg2154', 'ArRYEpsg2154', 'ZdAId', 'ArRGeopoint',
       'ArRTown', 'ArRPostalRegion', 'ArRAccessibility', 'ArRAudibleSignals',
       'ArRVisualSigns', 'ArRFareZone'],
      dtype='object')

### beaucoup de colonnes inutiles

In [6]:


# Supprimer les colonnes spécifiées
columns_to_drop = [
    'Rang', 'Correspondance_1', 'Correspondance_2', 'Correspondance_3', 
    'Correspondance_4', 'Correspondance_5', 'ArRId', 'ArRVersion', 
    'ArRCreated', 'ArRChanged', 'ArRName', 'ArRType', 'ArRPublicCode', 
    'ArRXEpsg2154', 'ArRYEpsg2154', 'ZdAId', 'ArRTown', 'ArRPostalRegion', 
    'ArRAccessibility', 'ArRAudibleSignals', 'ArRVisualSigns', 'ArRFareZone'
]
merged_df = merged_df.drop(columns=columns_to_drop)

# Afficher les premières lignes du DataFrame fusionné pour vérifier
print("Merged DataFrame sans les colonnes spécifiées:")
print(merged_df.head())

# Vérifier quelques valeurs spécifiques
sample_stations = ['opera', 'hotel de ville']  # Par exemple, en minuscules
sample_arrets = merged_df[merged_df['Station'].isin(sample_stations)]
print("Sample Stations DataFrame:")
print(sample_arrets)

# Enregistrer le DataFrame résultant dans un nouveau fichier CSV
output_file_path = 'final_merged_df_sans_colonnes_specifiees.csv'
merged_df.to_csv(output_file_path, index=False)

print(f"Les DataFrames ont été fusionnés et enregistrés dans {output_file_path}")


Merged DataFrame sans les colonnes spécifiées:
  Réseau Station   Trafic  Ville  Arrondissement pour Paris  Year  \
0  Métro  nation  9541406  Paris                       12.0  2013   
1  Métro  nation  9541406  Paris                       12.0  2013   
2  Métro  nation  9541406  Paris                       12.0  2013   
3  Métro  nation  9541406  Paris                       12.0  2013   
4  Métro  nation  9541406  Paris                       12.0  2013   

                              ArRGeopoint  
0   48.84845279706631, 2.3957988351125254  
1  48.848084289960305, 2.3958439887237373  
2   48.848453436965976, 2.395925559927623  
3  48.848885218403474, 2.3959437210498136  
4   48.84817470172562, 2.3959383298941117  
Sample Stations DataFrame:
Empty DataFrame
Columns: [Réseau, Station, Trafic, Ville, Arrondissement pour Paris, Year, ArRGeopoint]
Index: []
Les DataFrames ont été fusionnés et enregistrés dans final_merged_df_sans_colonnes_specifiees.csv


In [7]:
merged_dff = pd.read_csv('merged_dff.csv')
merged_dff

,Réseau,Station,Trafic,Ville,Arrondissement pour Paris,Year,ArRGeopoint
0,Métro,nation,9541406,Paris,12.0,2013,"48.84845279706631, 2.3957988351125254"
1,Métro,nation,9541406,Paris,12.0,2013,"48.848084289960305, 2.3958439887237373"
2,Métro,nation,9541406,Paris,12.0,2013,"48.848453436965976, 2.395925559927623"
3,Métro,nation,9541406,Paris,12.0,2013,"48.848885218403474, 2.3959437210498136"
4,Métro,nation,9541406,Paris,12.0,2013,"48.84817470172562, 2.3959383298941117"
...,...,...,...,...,...,...,...
16487,RER,parc de sceaux,383626,Antony,NaN,2021,"48.769885381618195, 2.309994431722374"
16488,RER,parc de sceaux,383626,Antony,NaN,2021,"48.77838909051428, 2.3029672849882807"
16489,RER,parc de sceaux,383626,Antony,NaN,2021,"48.769552376312525, 2.309956653435713"
16490,RER,parc de sceaux,383626,Antony,NaN,2021,"48.769569891830265, 2.3098747819817276"


In [8]:
###on clean au maximum le dataset afin de conserver que lessentiel ( on a peu de ressource de compute)

# Line Transport

In [27]:



# Charger les données JSON
file_path = 'traces-du-reseau-ferre-idf.json' 
voie_data = load_json(file_path)

# Extraire les informations des voies à partir du JSON
voies_list = []
for feature in voie_data['features']:
    properties = feature['properties']
    coordinates = feature['geometry']['coordinates']
    voie = {
        'idrefliga': properties['idrefliga'],
        'reseau': properties['reseau'],
        'mode': properties['mode'],
        'res_com': properties['res_com'],
        'coordinates': coordinates
    }
    voies_list.append(voie)

# Convertir en DataFrame
voies_df = pd.DataFrame(voies_list)

# Afficher les premières lignes du DataFrame
voies_df.head()

,idrefliga,reseau,mode,res_com,coordinates
0,A01761,TRAMWAY,TRAMWAY,TRAM 4,"[[2.487910986261173, 48.893094628632376], [2.4..."
1,A01842,RER D,RER,RER D,"[[2.45257339227245, 48.63426831236897], [2.451..."
2,A01842,RER D,RER,RER D,"[[2.435330530181304, 48.64848911258699], [2.43..."
3,A01842,RER D,RER,RER D,"[[2.397532093159732, 48.66791576646778], [2.39..."
4,A01842,RER D,RER,RER D,"[[2.407796286173906, 48.63635136710196], [2.40..."


on a un interer pour les nom de reseau, quon va merge dans notre dataframe

In [26]:


# Charger les DataFrames
merged_df = pd.read_csv("merged_dff.csv")

# Charger le fichier JSON
with open('traces-du-reseau-ferre-idf.json', 'r') as f:
    json_data = json.load(f)

# Extraire les propriétés et les coordonnées du JSON
features = json_data['features']
json_list = []
for feature in features:
    properties = feature['properties']
    coordinates = feature['geometry']['coordinates']
    res_com = properties.get('res_com', None)
    for coord in coordinates:
        properties_filtered = {'coordinate': coord, 'res_com': res_com}
        json_list.append(properties_filtered)

# Convertir la liste en DataFrame
json_df = pd.DataFrame(json_list)

# Convertir les geopoints en objets géométriques Point
merged_df['geometry'] = merged_df['ArRGeopoint'].apply(lambda x: Point(map(float, x.split(','))))
json_df['geometry'] = json_df['coordinate'].apply(lambda x: Point(x[1], x[0]))  # Inverser l'ordre des coordonnées

# Convertir les DataFrames en GeoDataFrames
gdf_merged = gpd.GeoDataFrame(merged_df, geometry='geometry')
gdf_json = gpd.GeoDataFrame(json_df, geometry='geometry')

# Définir la tolérance pour la comparaison des points géographiques
tolerance = 1e-3  # Ajustez cette tolérance selon vos besoins

# Effectuer une jointure spatiale en utilisant sjoin_nearest avec une tolérance
gdf_json.set_crs(epsg=4326, inplace=True)
gdf_merged.set_crs(epsg=4326, inplace=True)

# Ajouter une petite mise en mémoire tampon pour la tolérance
gdf_json['geometry'] = gdf_json['geometry'].buffer(tolerance)

# Effectuer la jointure spatiale
merged_gdf = gpd.sjoin_nearest(gdf_merged, gdf_json, how='left', max_distance=tolerance, distance_col='dist')

# Afficher les colonnes disponibles après la jointure spatiale
print("Colonnes disponibles après la jointure spatiale:")
print(merged_gdf.columns)

# Vérifier si 'res_com' existe et l'ajouter au DataFrame fusionné
if 'res_com' in merged_gdf.columns:
    merged_gdf['res_com'] = merged_gdf['res_com']
elif 'res_com_right' in merged_gdf.columns:
    merged_gdf['res_com'] = merged_gdf['res_com_right']
else:
    print("La colonne 'res_com' n'a pas été trouvée après la jointure.")

# Supprimer les colonnes temporaires et non nécessaires
merged_gdf = merged_gdf.drop(columns=['coordinate', 'geometry', 'dist', 'index_right'])

# Afficher les premières lignes du DataFrame fusionné pour vérifier
print("Final Merged DataFrame:")
print(merged_gdf.head())

# Enregistrer le DataFrame résultant dans un nouveau fichier CSV
output_file_path = 'final_merged_df_with_res_com.csv'
merged_gdf.to_csv(output_file_path, index=False)

print(f"Les DataFrames ont été fusionnés et enregistrés dans {output_file_path}")


C:\Users\SKONIECZNY\AppData\Local\Temp\ipykernel_816\3633532519.py:43: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_json['geometry'] = gdf_json['geometry'].buffer(tolerance)
c:\Users\SKONIECZNY\anaconda3\envs\BigPipe\Lib\site-packages\geopandas\array.py:365: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


Colonnes disponibles après la jointure spatiale:
Index(['Réseau', 'Station', 'Trafic', 'Ville', 'Arrondissement pour Paris',
       'Year', 'ArRGeopoint', 'geometry', 'index_right', 'coordinate',
       'res_com', 'dist'],
      dtype='object')
Final Merged DataFrame:
  Réseau Station   Trafic  Ville  Arrondissement pour Paris  Year  \
0  Métro  nation  9541406  Paris                       12.0  2013   
0  Métro  nation  9541406  Paris                       12.0  2013   
0  Métro  nation  9541406  Paris                       12.0  2013   
0  Métro  nation  9541406  Paris                       12.0  2013   
0  Métro  nation  9541406  Paris                       12.0  2013   

                             ArRGeopoint  res_com  
0  48.84845279706631, 2.3957988351125254    RER A  
0  48.84845279706631, 2.3957988351125254  METRO 2  
0  48.84845279706631, 2.3957988351125254  METRO 9  
0  48.84845279706631, 2.3957988351125254  METRO 1  
0  48.84845279706631, 2.3957988351125254  METRO 1  
Les 

In [27]:
df_final = pd.read_csv('final_merged_df_with_res_com.csv')
df_final

,Réseau,Station,Trafic,Ville,Arrondissement pour Paris,Year,ArRGeopoint,res_com
0,Métro,nation,9541406,Paris,12.0,2013,"48.84845279706631, 2.3957988351125254",RER A
1,Métro,nation,9541406,Paris,12.0,2013,"48.84845279706631, 2.3957988351125254",METRO 2
2,Métro,nation,9541406,Paris,12.0,2013,"48.84845279706631, 2.3957988351125254",METRO 9
3,Métro,nation,9541406,Paris,12.0,2013,"48.84845279706631, 2.3957988351125254",METRO 1
4,Métro,nation,9541406,Paris,12.0,2013,"48.84845279706631, 2.3957988351125254",METRO 1
...,...,...,...,...,...,...,...,...
102210,RER,parc de sceaux,383626,Antony,NaN,2021,"48.769569891830265, 2.3098747819817276",RER B
102211,RER,parc de sceaux,383626,Antony,NaN,2021,"48.769569891830265, 2.3098747819817276",RER B
102212,RER,parc de sceaux,383626,Antony,NaN,2021,"48.769569891830265, 2.3098747819817276",RER B
102213,RER,parc de sceaux,383626,Antony,NaN,2021,"48.769569891830265, 2.3098747819817276",RER B


# on va link les coordonnées au dataframe précédent

# on vérife les chantiers travaux


In [28]:
SNCF_df = pd.read_csv('./TravauxSNCF_RATP_2019-2022.csv', sep=';')
SNCF_df

,Référence Chantier,Code postal arrondissement - Commune,Date début du chantier,Date fin du chantier,Responsable du chantier,Maîtrise d'ouvrage principale,Surface (m2),Synthèse - Nature du chantier,Encombrement espace public,Impact stationnement,Identifiant demande CITE,Identifiant Chantier CITE,geo_shape,geo_point_2d
0,EC149360,75004,2019-12-16,2019-12-20,Opérateurs de réseau (gaz-électricité-RATP-etc),RATP,9.13,Travaux sur réseau de transport public,EMPRISE_TROTTOIR,NaN,130520,103482,"{""coordinates"": [[[2.3584960379852222, 48.8543...","48.85431161383518, 2.358496563308432"
1,EC143169,75015,2019-11-25,2019-12-13,Opérateurs de réseau (gaz-électricité-RATP-etc),RATP,224.69,Travaux sur réseau de transport public,"EMPRISE_CHAUSSEE,EMPRISE_TROTTOIR",NaN,124986,98918,"{""coordinates"": [[[2.298130063341984, 48.84908...","48.84904331680768, 2.2979730027487317"
2,EC146035,75017,2019-11-28,2019-12-06,Opérateurs de réseau (gaz-électricité-RATP-etc),SNCF,383.32,Travaux sur réseau de transport public,EMPRISE_CHAUSSEE,NaN,127442,100873,"{""coordinates"": [[[2.294098752608522, 48.87490...","48.875105790329215, 2.2938792856422676"
3,EC101353,75001,2019-03-08,2019-06-28,Opérateurs de réseau (gaz-électricité-RATP-etc),RATP,103.44,Travaux sur réseau de transport public,EMPRISE_TROTTOIR,NaN,87566,67973,"{""coordinates"": [[[[2.342282086238916, 48.8586...","48.85869275120488, 2.3421547092435926"
4,EC6520,75012,2016-01-01,2022-01-25,Opérateurs de réseau (gaz-électricité-RATP-etc),RATP,1760.89,Travaux sur réseau de transport public,EMPRISE_CHAUSSEE,NaN,4037,3079,"{""coordinates"": [[[2.396792756027481, 48.84758...","48.847861582585196, 2.3968969115602876"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2547,EC304653,75015,2022-10-06,2022-11-10,Opérateurs de réseau (gaz-électricité-RATP-etc),RATP,19.28,Travaux sur réseau de transport public,EMPRISE_TROTTOIR,NaN,733881,201349,"{""coordinates"": [[[2.284352185478997, 48.83391...","48.83388082847594, 2.2844072785931644"
2548,EC311629,75014,2022-11-24,2022-12-02,Opérateurs de réseau (gaz-électricité-RATP-etc),RATP,13.96,Travaux sur réseau de transport public,EMPRISE_CHAUSSEE,STATIONNEMENT_PAYANT,767173,205914,"{""coordinates"": [[[2.32495917057641, 48.841083...","48.84106036189037, 2.325000795252847"
2549,EC237774,75012,2021-10-11,2022-06-23,Opérateurs de réseau (gaz-électricité-RATP-etc),RATP,62.40,Travaux sur réseau de transport public,EMPRISE_TROTTOIR,NaN,445439,159992,"{""coordinates"": [[[[2.371750724396233, 48.8447...","48.844776005236575, 2.3717317530274373"
2550,EC302876,75020,2022-11-14,2023-06-08,Opérateurs de réseau (gaz-électricité-RATP-etc),RATP,201.35,Travaux sur réseau de transport public,"EMPRISE_CHAUSSEE,EMPRISE_TROTTOIR",NaN,727404,200226,"{""coordinates"": [[[[2.3963398106706215, 48.854...","48.85388041288721, 2.3964143861956844"


### on va recuperer les voies qui ont deja eu des travaux et merge avec notre dataframe, s'il en adeja eu , on lui accorde 1 , si jamais on lui accorde 0; et si cest reccurent on lui accorde 2

In [35]:

# Charger les DataFrames
df_final = pd.read_csv('final_merged_df_with_res_com.csv')

# Charger le fichier de travaux avec le bon délimiteur
df_travaux = pd.read_csv('TravauxSNCF_RATP_2019-2022.csv', delimiter=';')

# Convertir les coordonnées en objets géométriques Point pour df_final
df_final['geometry'] = df_final['ArRGeopoint'].apply(lambda x: Point(map(float, x.split(','))))

# Extraire les coordonnées de la colonne 'geo_shape' pour df_travaux
def extract_point(geo_shape):
    try:
        if isinstance(geo_shape, str):
            geo_json = json.loads(geo_shape.replace("'", "\""))
            if geo_json['type'] == 'Point':
                return Point(geo_json['coordinates'])
            elif geo_json['type'] == 'LineString':
                # Si c'est une ligne, nous prenons le premier point
                return Point(geo_json['coordinates'][0])
            elif geo_json['type'] == 'Polygon':
                # Si c'est un polygone, nous prenons le premier point du premier anneau
                return Point(geo_json['coordinates'][0][0])
        return None
    except Exception as e:
        print(f"Error parsing geo_shape: {e}")
        return None

df_travaux['geometry'] = df_travaux['geo_shape'].apply(extract_point)

# Filtrer les lignes où la géométrie a été correctement extraite
df_travaux = df_travaux.dropna(subset=['geometry'])

# Convertir les DataFrames en GeoDataFrames
gdf_final = gpd.GeoDataFrame(df_final, geometry='geometry')
gdf_travaux = gpd.GeoDataFrame(df_travaux, geometry='geometry')

# Définir le système de coordonnées
gdf_final.set_crs(epsg=4326, inplace=True)
gdf_travaux.set_crs(epsg=4326, inplace=True)

# Effectuer une jointure spatiale avec tolérance
joined_gdf = gpd.sjoin(gdf_final, gdf_travaux, how='left', predicate='intersects')

# Créer une nouvelle colonne `travaux` et initialiser à 0
joined_gdf['travaux'] = 0

# Mettre à jour la colonne `travaux` en fonction des occurrences de travaux
# Compter le nombre de fois qu'une station a eu des travaux
travaux_counts = joined_gdf.groupby(gdf_final.index).size()

# Mettre à jour la colonne `travaux` en fonction du nombre de travaux
for idx, count in travaux_counts.items():
    if count > 1:
        joined_gdf.at[idx, 'travaux'] = 2
    elif count == 1:
        joined_gdf.at[idx, 'travaux'] = 1

# Supprimer les colonnes temporaires et non nécessaires
joined_gdf = joined_gdf.drop(columns=['index_right', 'geometry'])

# # Enregistrer le DataFrame résultant dans un nouveau fichier CSV
# output_file_path = 'final_merged_df_with_travaux.csv'
# joined_gdf.to_csv(output_file_path, index=False)

# print(f"Les DataFrames ont été fusionnés et enregistrés dans {output_file_path}")

joined_gdf

,Réseau,Station,Trafic,Ville,Arrondissement pour Paris,Year,ArRGeopoint,res_com,Référence Chantier,Code postal arrondissement - Commune,...,Maîtrise d'ouvrage principale,Surface (m2),Synthèse - Nature du chantier,Encombrement espace public,Impact stationnement,Identifiant demande CITE,Identifiant Chantier CITE,geo_shape,geo_point_2d,travaux
0,Métro,nation,9541406,Paris,12.0,2013,"48.84845279706631, 2.3957988351125254",RER A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Métro,nation,9541406,Paris,12.0,2013,"48.84845279706631, 2.3957988351125254",METRO 2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Métro,nation,9541406,Paris,12.0,2013,"48.84845279706631, 2.3957988351125254",METRO 9,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Métro,nation,9541406,Paris,12.0,2013,"48.84845279706631, 2.3957988351125254",METRO 1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,Métro,nation,9541406,Paris,12.0,2013,"48.84845279706631, 2.3957988351125254",METRO 1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102210,RER,parc de sceaux,383626,Antony,NaN,2021,"48.769569891830265, 2.3098747819817276",RER B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
102211,RER,parc de sceaux,383626,Antony,NaN,2021,"48.769569891830265, 2.3098747819817276",RER B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
102212,RER,parc de sceaux,383626,Antony,NaN,2021,"48.769569891830265, 2.3098747819817276",RER B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
102213,RER,parc de sceaux,383626,Antony,NaN,2021,"48.769569891830265, 2.3098747819817276",RER B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


### observation

On a récupéré les travaux, on va pouvoir supprimer les colonnes inutiles

In [36]:
joined_gdf.columns

Index(['Réseau', 'Station', 'Trafic', 'Ville', 'Arrondissement pour Paris',
       'Year', 'ArRGeopoint', 'res_com', 'Référence Chantier',
       'Code postal arrondissement - Commune', 'Date début du chantier',
       'Date fin du chantier', 'Responsable du chantier',
       'Maîtrise d'ouvrage principale', 'Surface (m2)',
       'Synthèse - Nature du chantier', 'Encombrement espace public',
       'Impact stationnement', 'Identifiant demande CITE',
       'Identifiant Chantier CITE', 'geo_shape', 'geo_point_2d', 'travaux'],
      dtype='object')

In [37]:
# Supprimer les colonnes spécifiées
colonnes_a_supprimer = ['Référence Chantier',
       'Code postal arrondissement - Commune', 'Date début du chantier',
       'Date fin du chantier', 'Responsable du chantier',
       "Maîtrise d'ouvrage principale", 'Surface (m2)',
       'Synthèse - Nature du chantier', 'Encombrement espace public',
       'Impact stationnement', 'Identifiant demande CITE',
       'Identifiant Chantier CITE', 'geo_shape', 'geo_point_2d']

joined_gdf = joined_gdf.drop(columns=colonnes_a_supprimer)

# Enregistrer le DataFrame résultant dans un nouveau fichier CSV
output_file_path = 'final_merged_df_with_travaux.csv'
joined_gdf.to_csv(output_file_path, index=False)

print(f"Les DataFrames ont été fusionnés, les colonnes spécifiées ont été supprimées, et le résultat a été enregistré dans {output_file_path}")

Les DataFrames ont été fusionnés, les colonnes spécifiées ont été supprimées, et le résultat a été enregistré dans final_merged_df_with_travaux.csv


## NB Fer

In [65]:

# Chemins des fichiers CSV
file_paths = [
    r"C:\Users\SKONIECZNY\Desktop\Transport_App\ML_module\data_model\validation\merged_NB_FER.csv\part-00112-4bea7afa-2c52-475e-b6d2-caee215c049a-c000.csv",
    r"C:\Users\SKONIECZNY\Desktop\Transport_App\ML_module\data_model\validation\merged_NB_FER.csv\part-00113-4bea7afa-2c52-475e-b6d2-caee215c049a-c000.csv",
    r"C:\Users\SKONIECZNY\Desktop\Transport_App\ML_module\data_model\validation\merged_NB_FER.csv\part-00114-4bea7afa-2c52-475e-b6d2-caee215c049a-c000.csv",
    r"C:\Users\SKONIECZNY\Desktop\Transport_App\ML_module\data_model\validation\merged_NB_FER.csv\part-00115-4bea7afa-2c52-475e-b6d2-caee215c049a-c000.csv",
    r"C:\Users\SKONIECZNY\Desktop\Transport_App\ML_module\data_model\validation\merged_NB_FER.csv\part-00116-4bea7afa-2c52-475e-b6d2-caee215c049a-c000.csv",
    r"C:\Users\SKONIECZNY\Desktop\Transport_App\ML_module\data_model\validation\merged_NB_FER.csv\part-00117-4bea7afa-2c52-475e-b6d2-caee215c049a-c000.csv",
    r"C:\Users\SKONIECZNY\Desktop\Transport_App\ML_module\data_model\validation\merged_NB_FER.csv\part-00118-4bea7afa-2c52-475e-b6d2-caee215c049a-c000.csv",
    r"C:\Users\SKONIECZNY\Desktop\Transport_App\ML_module\data_model\validation\merged_NB_FER.csv\part-00119-4bea7afa-2c52-475e-b6d2-caee215c049a-c000.csv"
]

# Initialiser une liste pour stocker les DataFrames
dataframes = []

# Lire les fichiers CSV
for file_path in file_paths:
    df = pd.read_csv(file_path, delimiter=',')
    dataframes.append(df)

# Concaténer tous les DataFrames en un seul
df_concat = pd.concat(dataframes, ignore_index=True)

# Afficher les premières lignes du DataFrame concaténé pour vérifier
print(df_concat.head())

         JOUR  CODE_STIF_TRNS CODE_STIF_RES CODE_STIF_ARRET  LIBELLE_ARRET  \
0  01/07/2022             100           110               1  PORTE MAILLOT   
1  01/07/2022             100           110               1  PORTE MAILLOT   
2  01/07/2022             100           110               1  PORTE MAILLOT   
3  01/07/2022             100           110               1  PORTE MAILLOT   
4  01/07/2022             100           110               1  PORTE MAILLOT   

   ID_REFA_LDA CATEGORIE_TITRE  NB_VALD  date  
0        71379               ?      253  2022  
1        71379       AMETHYSTE      171  2022  
2        71379     AUTRE TITRE     1398  2022  
3        71379             FGT      247  2022  
4        71379       IMAGINE R     2510  2022  


#### pourquoi seulement quelques un

les données sont volumineuses et je n'ai pas les ressources afin de développer avec plus

In [67]:
df_concat.columns

Index(['JOUR', 'CODE_STIF_TRNS', 'CODE_STIF_RES', 'CODE_STIF_ARRET',
       'LIBELLE_ARRET', 'ID_REFA_LDA', 'CATEGORIE_TITRE', 'NB_VALD', 'date'],
      dtype='object')

## on cherche à voir laffluence des stations

pour cela nous conserverons seuleemnt les nombre de validation par jour , ferons laggregations par jour / mois et qualiefirons les stations à grande inffluence

In [70]:
# Supprimer les colonnes spécifiées
columns_to_drop = ['CODE_STIF_TRNS', 'CODE_STIF_RES', 'CODE_STIF_ARRET', 'ID_REFA_LDA', 'CATEGORIE_TITRE', 'date']
df_concat.drop(columns=columns_to_drop, inplace=True)

# Afficher les premières lignes du DataFrame concaténé pour vérifier
print(df_concat.head())

KeyError: "['CODE_STIF_TRNS', 'CODE_STIF_RES', 'CODE_STIF_ARRET', 'ID_REFA_LDA', 'CATEGORIE_TITRE', 'date'] not found in axis"

In [71]:
df_concat

,JOUR,LIBELLE_ARRET,NB_VALD
0,01/07/2022,PORTE MAILLOT,253
1,01/07/2022,PORTE MAILLOT,171
2,01/07/2022,PORTE MAILLOT,1398
3,01/07/2022,PORTE MAILLOT,247
4,01/07/2022,PORTE MAILLOT,2510
...,...,...,...
1105942,31/12/2022,VILLEPINTE,212
1105943,31/12/2022,VILLEPINTE,614
1105944,31/12/2022,VILLEPINTE,6
1105945,31/12/2022,VILLEPINTE,6


l'objectif est de calculer l'affluence , alors on va regrouper par jour chaque arret et determiner l'affluence total de la journée

In [73]:
# Convertir les dates en datetime si nécessaire
df_concat['JOUR'] = pd.to_datetime(df_concat['JOUR'], format='%d/%m/%Y', errors='coerce')

# Convertir les noms de stations en minuscules pour une correspondance uniforme
df_concat['LIBELLE_ARRET'] = df_concat['LIBELLE_ARRET'].str.lower()

# Grouper par JOUR et LIBELLE_ARRET, puis sommer les valeurs de NB_VALD
df_grouped = df_concat.groupby(['JOUR', 'LIBELLE_ARRET'], as_index=False).agg({'NB_VALD': 'sum'})

In [74]:
df_grouped

,JOUR,LIBELLE_ARRET,NB_VALD
0,2022-07-01,abbesses,4011
1,2022-07-01,ablon,1099
2,2022-07-01,acheres grand,53
3,2022-07-01,acheres ville,4009
4,2022-07-01,aeroport cdg1,5269
...,...,...,...
135734,2022-12-31,voltaire,4305
135735,2022-12-31,vulaines sur se,23
135736,2022-12-31,wagram,1395
135737,2022-12-31,yerres,1546


on peut alors merge nos dataframes

In [77]:
df_final = pd.read_csv('final_merged_df_with_travaux.csv')

# Fusionner les deux DataFrames
merged_df = pd.merge(df_final, df_grouped, left_on='Station', right_on='LIBELLE_ARRET', how='left')

# Afficher les premières lignes du DataFrame fusionné pour vérifier
print("Merged DataFrame:")
print(merged_df.head())

Merged DataFrame:
  Réseau Station   Trafic  Ville  Arrondissement pour Paris  Year  \
0  Métro  nation  9541406  Paris                       12.0  2013   
1  Métro  nation  9541406  Paris                       12.0  2013   
2  Métro  nation  9541406  Paris                       12.0  2013   
3  Métro  nation  9541406  Paris                       12.0  2013   
4  Métro  nation  9541406  Paris                       12.0  2013   

                             ArRGeopoint res_com  travaux       JOUR  \
0  48.84845279706631, 2.3957988351125254   RER A        1 2022-07-01   
1  48.84845279706631, 2.3957988351125254   RER A        1 2022-07-02   
2  48.84845279706631, 2.3957988351125254   RER A        1 2022-07-03   
3  48.84845279706631, 2.3957988351125254   RER A        1 2022-07-04   
4  48.84845279706631, 2.3957988351125254   RER A        1 2022-07-05   

  LIBELLE_ARRET  NB_VALD  
0        nation  32686.0  
1        nation  24486.0  
2        nation  17702.0  
3        nation  31065.0  

In [78]:
merged_df.columns

Index(['Réseau', 'Station', 'Trafic', 'Ville', 'Arrondissement pour Paris',
       'Year', 'ArRGeopoint', 'res_com', 'travaux', 'JOUR', 'LIBELLE_ARRET',
       'NB_VALD'],
      dtype='object')

In [81]:
output_file_path = 'df_nb_valid_trav_res.csv'
merged_df.to_csv(output_file_path, index=False)

print(f"Le DataFrame groupé a été enregistré dans {output_file_path}")

Le DataFrame groupé a été enregistré dans df_nb_valid_trav_res.csv


### Il manque juste la météo

In [ ]:
weather_df = pd.read_csv('./all_weather/2023-2024-weather_cleaned.csv')
weather_df

: 

In [86]:
weather_df.columns

Index(['NUM_POSTE', 'NOM_USUEL', 'LAT', 'LON', 'ALTI_x', 'AAAAMMJJ', 'PMERM',
       'QPMERM', 'PMERMIN', 'QPMERMIN', 'INST', 'QINST', 'GLOT', 'QGLOT',
       'SIGMA', 'QSIGMA', 'UN', 'QUN', 'HUN', 'QHUN', 'UX', 'QUX', 'HUX',
       'QHUX', 'UM', 'QUM', 'DHUMI40', 'QDHUMI40', 'DHUMI80', 'QDHUMI80',
       'TSVM', 'QTSVM', 'ETPMON', 'QETPMON', 'ETPGRILLE', 'QETPGRILLE',
       'NEIGETOTX', 'QNEIGETOTX', 'NEIGETOT06', 'QNEIGETOT06', 'NEIG', 'QNEIG',
       'BROU', 'QBROU', 'GELEE', 'QGELEE', 'FUMEE', 'QFUMEE', 'BRUME',
       'QBRUME', 'ALTI_y', 'RR', 'QRR', 'TN', 'QTN', 'HTN', 'QHTN', 'TX',
       'QTX', 'HTX', 'QHTX', 'TM', 'QTM', 'TNTXM', 'QTNTXM', 'TAMPLI',
       'QTAMPLI', 'TNSOL', 'QTNSOL', 'TN50', 'QTN50', 'DG', 'QDG', 'FFM',
       'QFFM', 'FXY', 'QFXY', 'DXY', 'QDXY', 'HXY', 'QHXY', 'FXI', 'QFXI',
       'DXI', 'QDXI', 'HXI', 'QHXI', 'FXI3S', 'QFXI3S', 'QDXI3S', 'HXI3S',
       'QHXI3S'],
      dtype='object')

je garde ceux qui ont le plus d'impact : 


PMERM - Mean precipitation
TX - Maximum temperature
TN - Minimum temperature
HTX - High temperature
RR - Rainfall
NEIG - Snow
BRUME - Fog
GELEE - Frost
BROU - Mist
GLOT - Global radiation

In [ ]:
# Convert to GeoDataFrames
merged_df['geometry'] = merged_df['ArRGeopoint'].apply(lambda x: Point(map(float, x.split(','))))
gdf_merged = gpd.GeoDataFrame(merged_df, geometry='geometry')

weather_df['geometry'] = weather_df.apply(lambda row: Point(row['LON'], row['LAT']), axis=1)
gdf_weather = gpd.GeoDataFrame(weather_df, geometry='geometry')

# Ensure both GeoDataFrames use the same CRS
gdf_merged.set_crs(epsg=4326, inplace=True)
gdf_weather.set_crs(epsg=4326, inplace=True)

# Define the number of chunks
num_chunks = 100  # Adjust this number based on your memory capacity

# Split the merged DataFrame into smaller chunks
chunks = np.array_split(gdf_merged, num_chunks)

# List to hold the results
results = []

# Perform spatial join for each chunk
for chunk in chunks:
    joined_chunk = gpd.sjoin_nearest(chunk, gdf_weather, how='left', distance_col='dist')
    results.append(joined_chunk)

# Concatenate all chunks into a single GeoDataFrame
final_gdf = pd.concat(results, ignore_index=True)

# Select relevant columns from the weather DataFrame to merge
weather_columns = [
    'PMERM', 'TX', 'TN', 'HTX', 'RR', 'NEIG', 'BRUME', 'GELEE', 'BROU', 'GLOT'
]

# Include weather columns in the final DataFrame
columns_to_keep = ['Réseau', 'Station', 'Trafic', 'Ville', 'Arrondissement pour Paris', 'Year', 
                   'ArRGeopoint', 'res_com', 'travaux', 'JOUR', 'LIBELLE_ARRET', 'NB_VALD'] + weather_columns

final_gdf = final_gdf[columns_to_keep]

# Save the resulting DataFrame to a CSV file
output_file_path = 'final_merged_df_with_weather.csv'
final_gdf.to_csv(output_file_path, index=False)

print(f"The DataFrame has been merged and saved to {output_file_path}")

: 